In [ ]:
# 새로운거 빈 도메인만..

!python hhhhh.py multiple-fail

python: can't open file '/Users/jinym/Desktop/Desktop_AICenter✨/SFAIcenter/hhhhh.py': [Errno 2] No such file or directory


### multiple options 오류들
- find_multiple_choice_invalid_options.py

### 원본에 옮기기

In [ ]:
# import os, time
# from tqdm import tqdm

# file_list = []

# for t in tqdm(to_multiple):
#     file_id = t.get('file_id')
#     file_list.append(file_id)
#     # print(file_id)
#     tag = t.get('tag')
#     time.sleep(0.5)
        
#     file_path = os.popen(f"find /Users/jinym/Desktop/Desktop_AICenter✨/SFAIcenter/data/FINAL -type f -name '{file_id}_v2.json'").read().strip()
#     if file_path == "":
#         file_path = os.popen(f"find /Users/jinym/Desktop/Desktop_AICenter✨/SFAIcenter/data/FINAL -type f -name '{file_id}.json'").read().strip()
#     # print(file_path)

#     with open(file_path, 'r', encoding='utf-8') as f:
#         data = json.load(f)
    
#     contents = data['contents']

#     for d in contents:
#         if d.get('page') == tag.split('_')[1]:
#             add_info = d.get('add_info')
#             for info in add_info:
#                 if info.get('tag') == tag:
#                     if info.get('description').get('question') == t.get('question'):
#                         pass
#                     else:
#                         # print("질문 다름")
#                         info['description']['question'] = t.get('question')

#                     if info.get('description').get('answer') == t.get('answer'):
#                         pass
#                     else:
#                         # print("답 다름")
#                         info['description']['answer'] = t.get('answer')
#                     if info.get('description').get('options') == t.get('options'):
#                         pass
#                     else:
#                         # print("옵션 다름")
#                         info['description']['options'] = t.get('options')
#     if file_path.endswith("_v2.json"):
#         with open(file_path, 'w', encoding='utf-8') as f:
#             json.dump(data, f, ensure_ascii=False, indent=4)
#     else:
#         with open(file_path.replace(".json", "_v2.json"), 'w', encoding='utf-8') as f:
#             json.dump(data, f, ensure_ascii=False, indent=4)

In [ ]:
# sorted(list(set(file_list)))

## 통계

In [ ]:
import os, json
from tools.qna.processing.qna_subdomain_classifier import QnASubdomainClassifier
from tools.pipeline.config import ONEDRIVE_PATH

# file_name = os.path.join(ONEDRIVE_PATH, 'evaluation/eval_data/2_subdomain/multiple_subdomain_classified_ALL.json')
file_name = os.path.join(ONEDRIVE_PATH, 'evaluation', 'eval_data', '2_subdomain', 'multiple_subdomain_classified_ALL.json')
with open(file_name, 'r', encoding='utf-8') as f:
    multiple = json.load(f)

classifier = QnASubdomainClassifier()
classifier.save_statistics(multiple)

In [ ]:
# 도메인별 통계 계산
from collections import defaultdict



calculation = defaultdict(int)  # 계산 문제 개수
total_by_domain = defaultdict(int)  # 전체 문제 개수

for question in fifth_exam:
    domain = question.get('domain', '미분류')
    total_by_domain[domain] += 1
    if question.get('is_calculation') == True:
        calculation[domain] += 1

# 결과 출력
print("=" * 60)
print("도메인별 통계")
print("=" * 60)
print(f"{'도메인':<20} {'전체':<10} {'계산문제':<10} {'비율':<10}")
print("-" * 60)

for domain in sorted(total_by_domain.keys()):
    total = total_by_domain[domain]
    calc_count = calculation[domain]
    ratio = (calc_count / total * 100) if total > 0 else 0
    print(f"{domain:<20} {total:<10} {calc_count:<10} {ratio:.2f}%")

print("-" * 60)
print(f"{'합계':<20} {sum(total_by_domain.values()):<10} {sum(calculation.values()):<10} {sum(calculation.values())/sum(total_by_domain.values())*100:.2f}%")
print("=" * 60)

In [ ]:
python tools/main_pipeline.py --steps evaluate_exams --eval_sets 5 --eval_models google/gemini-2.5-pro openai/gpt-5 anthropic/claude-sonnet-4.5 google/gemini-2.5-flash anthropic/claude-3.7-sonnet openai/gpt-4.1

## evaluation 파일 수정하고 다시 돌리기

In [ ]:
import re
import os, json
from tools.pipeline.config import ONEDRIVE_PATH

EXAM_DIR = os.path.join(ONEDRIVE_PATH, 'evaluation','eval_data','4_multiple_exam')
# EXAM_DIR = os.path.join(os.path.expanduser("~"), '4_multiple_exam')

for exams in os.listdir(EXAM_DIR):
    # if exams == "1st":
    #     first_exam = []
    #     for exam in os.listdir(os.path.join(EXAM_DIR, exams)):
    #         with open(os.path.join(EXAM_DIR, exams, exam), 'r', encoding='utf-8') as f:
    #             first_exam += json.load(f)
    # elif exams == "2nd":
    #     second_exam = []
    #     for exam in os.listdir(os.path.join(EXAM_DIR, exams)):
    #         with open(os.path.join(EXAM_DIR, exams, exam), 'r', encoding='utf-8') as f:
    #             second_exam += json.load(f)
    # elif exams == "3rd":
    #     third_exam = []
    #     for exam in os.listdir(os.path.join(EXAM_DIR, exams)):
    #         with open(os.path.join(EXAM_DIR, exams, exam), 'r', encoding='utf-8') as f:
    #             third_exam += json.load(f)
    # elif exams == "4th":
    #     fourth_exam = []
    #     for exam in os.listdir(os.path.join(EXAM_DIR, exams)):
    #         with open(os.path.join(EXAM_DIR, exams, exam), 'r', encoding='utf-8') as f:
    #             fourth_exam += json.load(f)
    if exams == "5th":
        fifth_exam = []
        for exam in os.listdir(os.path.join(EXAM_DIR, exams)):
            with open(os.path.join(EXAM_DIR, exams, exam), 'r', encoding='utf-8') as f:
                fifth_exam += json.load(f)

# multiple = first_exam + second_exam + third_exam + fourth_exam + fifth_exam

In [6]:
import pandas as pd
import os
from tools.pipeline.config import ONEDRIVE_PATH

model_name = 'many'
excel_path = os.path.join(ONEDRIVE_PATH, f'evaluation/eval_data/6_exam_evaluation/5th/5th_evaluation_{model_name}.xlsx')
print(excel_path)
pred_wide = pd.read_excel(excel_path, sheet_name='모델별예측')

/Users/jinym/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/evaluation/eval_data/6_exam_evaluation/5th/5th_evaluation_many.xlsx


In [7]:
empty_ids = pred_wide[pred_wide['google/gemini-2.5-pro'].isna()].id.tolist()
# print(pred_wide.columns[-1])
print(empty_ids)

[]


In [ ]:
new_json = []
for q in empty_ids:
    for m in fifth_exam:
        if m['file_id']+"_"+m['tag'] == q:
            # print(m)
            new_json.append(m)

print(len(new_json) == len(empty_ids))

In [ ]:
import json
with open(os.path.join(ONEDRIVE_PATH, 'evaluation/model_new.json'), 'w', encoding='utf-8') as f:
    json.dump(new_json, f, ensure_ascii=False, indent=4)

In [8]:
# Excel 파일에서 정확도 계산 (전체, subject별, domain별, subdomain별)
import pandas as pd
import numpy as np
from typing import Set
import ast

# Excel 파일 경로 설정
# excel_path = f'/Users/yejin/Downloads/2nd_evaluation_ALL.xlsx'

# Excel 파일 읽기
df_sample = pd.read_excel(excel_path, sheet_name='전체데이터')
pred_wide = pd.read_excel(excel_path, sheet_name='모델별예측')

# pred_wide를 pred_long 형식으로 변환 (melt 사용)
# pred_wide는 id 컬럼과 각 모델명이 컬럼으로 있는 와이드 포맷
# pred_long은 id, model_name, answer 컬럼이 있는 롱 포맷
pred_long = pred_wide.melt(
    id_vars=['id'],  # id 컬럼은 그대로 유지
    var_name='model_name',  # 모델명 컬럼 이름
    value_name='answer'  # 예측값 컬럼 이름
)

# answer_set을 Set[int]로 변환하는 헬퍼 함수
def _parse_answer_set(answer_set):
    if isinstance(answer_set, set):
        return answer_set
    elif isinstance(answer_set, str):
        try:
            # 문자열을 파싱해서 Set[int]로 변환
            parsed = ast.literal_eval(answer_set)
            if isinstance(parsed, set):
                return parsed
            elif isinstance(parsed, (list, tuple)):
                return set(parsed)
            else:
                return {int(parsed)}
        except:
            # 파싱 실패 시 빈 집합 반환
            return set()
    else:
        # 숫자나 다른 타입인 경우
        try:
            return {int(answer_set)}
        except:
            return set()

# 정답 여부 계산 함수
def _is_correct(pred: float, s: Set[int]) -> float:
    if np.isnan(pred) or not s:
        return np.nan
    return float(int(pred) in s)

# 데이터 병합 및 정확도 계산
key = df_sample[["id", "answer_set"]].copy()
merged = pred_long.merge(key, on="id", how="left")
merged["answer_set"] = merged["answer_set"].apply(_parse_answer_set)
merged["correct"] = merged.apply(lambda r: _is_correct(r["answer"], r["answer_set"]), axis=1)

# 전체 정확도 계산
acc_by_model = (
    merged.groupby("model_name", dropna=False)["correct"]
    .mean()
    .reset_index()
    .rename(columns={"correct": "accuracy"})
    .sort_values("accuracy", ascending=False)
)

# Excel 파일에 결과 저장
# 기존 파일을 읽어서 모든 시트를 유지하면서 특정 시트만 업데이트
from openpyxl import load_workbook

# 기존 파일의 모든 시트 읽기 (업데이트할 시트 제외)
existing_sheets = {}
try:
    book = load_workbook(excel_path)
    for sheet_name in book.sheetnames:
        if sheet_name not in ["전체데이터", "모델별예측", "정확도", "Subject별정확도", "Subject별문제수", 
                              "Domain별정확도", "Domain별문제수", "Subdomain별정확도", "Subdomain별문제수"]:
            # 업데이트할 시트가 아닌 경우 기존 데이터 읽기
            existing_sheets[sheet_name] = pd.read_excel(excel_path, sheet_name=sheet_name)
except:
    pass

# 모든 시트를 새 파일로 저장
with pd.ExcelWriter(excel_path, engine="openpyxl", mode='w') as writer:
    # 기존 시트 유지
    for sheet_name, df in existing_sheets.items():
        df.to_excel(writer, index=False, sheet_name=sheet_name)
    # 기본 시트들 업데이트
    df_sample.to_excel(writer, index=False, sheet_name="전체데이터")
    pred_wide.to_excel(writer, index=False, sheet_name="모델별예측")
    acc_by_model.to_excel(writer, index=False, sheet_name="정확도")
    
    # subject별 정확도 계산 및 저장 (subject 컬럼이 있는 경우)
    if 'subject' in df_sample.columns:
        subject_info = df_sample[["id", "subject"]].copy()
        merged_with_subject = merged.merge(subject_info, on="id", how="left")
        
        subject_acc = (
            merged_with_subject.groupby(["subject", "model_name"], dropna=False)["correct"]
            .mean()
            .reset_index()
            .pivot(index="subject", columns="model_name", values="correct")
            .reset_index()
        )
        
        subject_acc.to_excel(writer, index=False, sheet_name="Subject별정확도")
        
        # Subject별 문제 수 통계
        subject_stats = df_sample.groupby('subject').size().reset_index(name='question_count')
        subject_stats.to_excel(writer, index=False, sheet_name="Subject별문제수")
    
    # domain별 정확도 계산 및 저장 (domain 컬럼이 있는 경우)
    if 'domain' in df_sample.columns:
        domain_info = df_sample[["id", "domain"]].copy()
        merged_with_domain = merged.merge(domain_info, on="id", how="left")
        
        domain_acc = (
            merged_with_domain.groupby(["domain", "model_name"], dropna=False)["correct"]
            .mean()
            .reset_index()
            .pivot(index="domain", columns="model_name", values="correct")
            .reset_index()
        )
        
        domain_acc.to_excel(writer, index=False, sheet_name="Domain별정확도")
        
        # Domain별 문제 수 통계
        domain_stats = df_sample.groupby('domain').size().reset_index(name='question_count')
        domain_stats.to_excel(writer, index=False, sheet_name="Domain별문제수")
    
    # subdomain별 정확도 계산 및 저장 (subdomain 컬럼이 있는 경우)
    if 'subdomain' in df_sample.columns:
        subdomain_info = df_sample[["id", "domain", "subdomain"]].copy()
        merged_with_subdomain = merged.merge(subdomain_info, on="id", how="left")
        
        subdomain_acc = (
            merged_with_subdomain.groupby(["domain", "subdomain", "model_name"], dropna=False)["correct"]
            .mean()
            .reset_index()
            .pivot(index=["domain", "subdomain"], columns="model_name", values="correct")
            .reset_index()
        )
        
        subdomain_acc.to_excel(writer, index=False, sheet_name="Subdomain별정확도")
        
        # Subdomain별 문제 수 통계
        subdomain_stats = df_sample.groupby(['domain', 'subdomain']).size().reset_index(name='question_count')
        subdomain_stats.to_excel(writer, index=False, sheet_name="Subdomain별문제수")

print(f"✅ Excel 파일 업데이트 완료: {excel_path}")

✅ Excel 파일 업데이트 완료: /Users/jinym/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/evaluation/eval_data/6_exam_evaluation/5th/5th_evaluation_many.xlsx
